<a href="https://colab.research.google.com/github/ii200400/Tensorflow_Tutorial/blob/master/05%20-%20TensorBoard%2C%20Saver/01_Saver(2_1ver).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

이번 실습은 구글 드라이브를 연동하여 파일을 저장하고 불러올 필요가 있기 때문에 아래의 코드를 추가하고 data.scv의 파일 경로를 바꿔주었다.

In [ ]:
# 구글 드라이브 연동을 위한 import
from google.colab import drive

# 구글 드라이브 연동을 위한 인증
drive.mount('/content/drive')

basic_path = 'drive/My Drive/Colab Notebooks/3분 딥러닝 텐서플로맛/05 - TensorBoard, Saver'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 개요

학습이 된 모델을 저장하고 재사용을 위해서 불러오기

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np

TensorFlow 2.x selected.


## 데이터 정의

다른 때와 다르게 데이터를 scv파일에서 불러온다.
csv는 입력값 저장하는 파일형식으로 자주쓰이는 것 같다.

또한 여기에서는 numpy 라이브러리와 여러 매개변수로 데이터를 가져왔지만
나중에는 더 편리한 라이브러리로 매개변수 없이도 쉽게 데이터를 조작할 것이다.

In [ ]:
# np.loadtxt의 매개변수 unpack과 np.transpose는
# 주어진 행렬을 전치행렬로 만들어 반환한다.

data = np.loadtxt(basic_path + '/data.csv', delimiter=',', unpack=True, dtype='float32')

# 털, 날개, 기타, 포유류, 조류
# x_data = 0, 1
# y_data = 2, 3, 4
x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])

## 모델 구성

### 가중치 정의

In [ ]:
W1 = tf.Variable(tf.random.uniform([2, 10], -1., 1.))
W2 = tf.Variable(tf.random.uniform([10, 20], -1., 1.))
W3 = tf.Variable(tf.random.uniform([20, 3], -1., 1.))

### 모델과 비용함수 구성

In [ ]:
def model(x):
  L1 = tf.nn.relu(tf.matmul(x, W1))
  L2 = tf.nn.relu(tf.matmul(L1, W2))

  return tf.matmul(L2, W3)

def get_cost(x, y):
  return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=model(x)))

### 모델 학습 함수

In [ ]:
def train(x, y):
  with tf.GradientTape() as t:
    cost = get_cost(x, y)
  
  var_list = [W1, W2, W3]
  grads = t.gradient(cost, var_list)
  optimizer.apply_gradients(zip(grads, var_list))

  return cost

### 최적화 함수

In [ ]:
# 최적화 함수는 아담을 사용한다.
optimizer = tf.optimizers.Adam(learning_rate=0.01)

## 모델 학습 및 테스트

### 모델 불러오기

텐서플로 1.0은 변수 기반의 체크포인트를 사용하지만\
텐서플로 2.0은 객체 기반의 체크포인트를 사용한다.

그래서 어떤 차이가 있는지는 잘 모르겠다;;

또한 나중에는 모델을 tf.keras.Sequential라는 텐서플로 기능을 활용할 것인데\
해당 기능을 사용하면 tf.saved_model.save라는 함수 하나로 쉽게 저장이 가능하다..

즉, 아래의 코드는 나중에 사용하지 않는다;;

In [ ]:
saver = tf.train.Checkpoint()

# model파일에 저장된 체크포인트가 있다면
ckpt = tf.train.get_checkpoint_state(basic_path + '/model(2.1ver)')
if ckpt:
  # 그 체크포인트를 불러온다.
  saver.restore(ckpt.model_checkpoint_path)

### 모델 학습 및 저장

In [ ]:
num = 2

for step in range(num):
  cost = train(x_data, y_data)

  print('Step: %d, ' % (saver.save_counter * num + step),
        'Cost: %.3f' % cost)
    
saver.save(basic_path + '/model(2.1ver)/dnn.ckpt')

Step: 4,  Cost: 1.530
Step: 5,  Cost: 1.426


'drive/My Drive/Colab Notebooks/3분 딥러닝 텐서플로맛/05 - TensorBoard, Saver/model(2.1ver)/dnn.ckpt-3'

### 모델 테스트

In [ ]:
# 결과 확인
# 0: 기타 1: 포유류, 2: 조류
prediction = tf.argmax(model(x_data), 1)
target = tf.argmax(y_data, 1)
print('예측값:', prediction)
print('실제값:', target)

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % (accuracy * 100))

예측값: tf.Tensor([0 1 2 0 0 0], shape=(6,), dtype=int64)
실제값: tf.Tensor([0 1 2 0 0 2], shape=(6,), dtype=int64)
정확도: 83.33


# 전체 코드

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np

data = np.loadtxt(basic_path + '/data.csv', delimiter=',', unpack=True, dtype='float32')

# 털, 날개, 기타, 포유류, 조류
# x_data = 0, 1
# y_data = 2, 3, 4
x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])

#########
# 신경망 모델 구성
######

W1 = tf.Variable(tf.random.uniform([2, 10], -1., 1.))
W2 = tf.Variable(tf.random.uniform([10, 20], -1., 1.))
W3 = tf.Variable(tf.random.uniform([20, 3], -1., 1.))

def model(x):
  L1 = tf.nn.relu(tf.matmul(x, W1))
  L2 = tf.nn.relu(tf.matmul(L1, W2))

  return tf.matmul(L2, W3)

def get_cost(x, y):
  return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=model(x)))

def train(x, y):
  with tf.GradientTape() as t:
    cost = get_cost(x, y)
  
  var_list = [W1, W2, W3]
  grads = t.gradient(cost, var_list)
  optimizer.apply_gradients(zip(grads, var_list))

  return cost

optimizer = tf.optimizers.Adam(learning_rate=0.01)

#########
# 신경망 모델 학습
######

saver = tf.train.Checkpoint()

ckpt = tf.train.get_checkpoint_state(basic_path + '/model(2.1ver)')
if ckpt:
  saver.restore(ckpt.model_checkpoint_path)

num = 2
for step in range(num):
  cost = train(x_data, y_data)

  print('Step: %d, ' % saver.save_counter * num + step + 1,
        'Cost: %.3f' % cost)
    
saver.save(basic_path + '/model(2.1ver)/dnn.ckpt')

#########
# 결과 확인
# 0: 기타 1: 포유류, 2: 조류
######

prediction = tf.argmax(model(x_data), 1)
target = tf.argmax(y_data, 1)
print('예측값:', prediction)
print('실제값:', target)

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % (accuracy * 100))

TensorFlow 2.x selected.
Step: 0,  Cost: 1.517
Step: 0,  Cost: 1.410
예측값: tf.Tensor([0 2 2 0 0 1], shape=(6,), dtype=int64)
실제값: tf.Tensor([0 1 2 0 0 2], shape=(6,), dtype=int64)
정확도: 66.67
